<a href="https://colab.research.google.com/github/angyf/proyecto/blob/main/Mapas_Coropl%C3%A9ticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mapas Coropléticos

In [ ]:
#Se requiere instalar geopandas para crear un solo dataframe con el geojson y los datos de interés para las funciones de resalte
!pip install geopandas

In [ ]:
#Se importan las librerías necesarias
import pandas as pd
import folium
import geopandas as gpd

In [ ]:
#Se guarda el link del geojson
state_geo ='https://raw.githubusercontent.com/angyf/proyecto/main/mexico.json'

In [ ]:
#Se lee y guarda el geojson
data_geo = gpd.read_file(state_geo)
data_geo.head()

,adm1_code,OBJECTID_1,diss_me,adm1_cod_1,iso_3166_2,wikipedia,iso_a2,adm0_sr,name,name_alt,name_local,type,type_en,code_local,code_hasc,note,hasc_maybe,region,region_cod,provnum_ne,gadm_level,check_me,scalerank,datarank,abbrev,postal,area_sqkm,sameascity,labelrank,featurecla,name_len,mapcolor9,mapcolor13,fips,fips_alt,woe_id,woe_label,woe_name,latitude,longitude,sov_a3,adm0_a3,adm0_label,admin,geonunit,gu_a3,gn_id,gn_name,gns_id,gns_name,gn_level,gn_region,gn_a1_code,region_sub,sub_code,gns_level,gns_lang,gns_adm1,gns_region,geometry
0,MEX+99?,3622,10110,MEX+99?,MX-,,MX,6,,,,,,,-99,MEX-99 (Mexico minor island),,,,0,0,20,11,11,,,0,-99,20,Admin-1 minor island,0,7,3,,,-99,,,22.3858,-89.6633,MEX,MEX,2,Mexico,Mexico,MEX,0,,0,,0,,MX.,,,0,,,,"POLYGON ((-89.69261 22.56707, -89.62492 22.497..."
1,MEX-2706,745,2706,MEX-2706,MX-,,MX,6,Baja California,,,Estado,State,,MX.BN,,,,,20006,1,20,4,2,,BC,0,-99,4,Admin-1 scale rank,15,7,3,MX02,,2346265,"Baja California, MX, Mexico",Baja California,30.3399,-115.2090,MEX,MEX,2,Mexico,Mexico,MEX,4017700,Estado de Baja California,-1652188,Baja California,1,,MX.02,,,1,zho,MX02,,"MULTIPOLYGON (((-114.14750 28.00015, -114.2070..."
2,MEX-2707,732,2707,MEX-2707,MX-,,MX,6,Baja California Sur,,,Estado,State,,MX.BS,,,,,20005,1,20,4,2,,BS,0,-99,4,Admin-1 scale rank,19,7,3,MX03,,2346266,"Baja California Sur, MX, Mexico",Baja California Sur,25.0656,-111.4520,MEX,MEX,2,Mexico,Mexico,MEX,4017698,Estado de Baja California Sur,-1652190,"Baja California Sur, Estado de",1,,MX.03,,,1,zho,MX03,,"MULTIPOLYGON (((-109.79843 24.15516, -109.8624..."
3,MEX-2708,2926,2708,MEX-2708,MX-,,MX,1,Coahuila,,,Estado,State,,MX.CA,,,,,20032,1,20,4,2,,CO,0,-99,4,Admin-1 scale rank,8,7,3,MX07,,2346270,"Coahuila de Zaragoza, MX, Mexico",Coahuila,27.4473,-101.8850,MEX,MEX,2,Mexico,Mexico,MEX,4013674,Estado de Coahuila de Zaragoza,-1658164,"Coahuila de Zaragoza, Estado de",1,,MX.07,,,1,zho,MX07,,"POLYGON ((-99.80696 27.77142, -99.96720 27.640..."
4,MEX-2709,2927,2709,MEX-2709,MX-,,MX,1,Chihuahua,,,Estado,State,,MX.CH,,,,,20034,1,20,4,2,,CH,0,6,6,Admin-1 scale rank,9,7,3,MX06,MX05,2346269,"Chihuahua, MX, Mexico",Chihuahua,28.9730,-106.2320,MEX,MEX,2,Mexico,Mexico,MEX,4014336,Estado de Chihuahua,-1657144,"Chihuahua, Estado de",1,,MX.06,,,1,zho,MX06,,"POLYGON ((-106.50605 31.77026, -106.42895 31.7..."


In [ ]:
#Se cambia la columna del nombre del estado (name) para que coincida con el nombre del data frame con los datos de interés
data_geo = data_geo.rename(columns={'name':'Entidad'})

#Mapa Tasa de Obesidad

In [ ]:
#Se lee el cvs con los datos de interés y se cambia el nombre de los estados para que coincidan con los nombres en el geojson
state_data=pd.read_csv('https://raw.githubusercontent.com/angyf/proyecto/main/Tasa_de_Obesidad_2018.csv')
state_data=state_data.replace({"Coahuila de Zaragoza": "Coahuila","Michoacán de Ocampo":"Michoacán","Veracruz de Ignacio de la Llave":"Veracruz"})
state_data

,Entidad,Tasa de obesidad
0,Estados Unidos Mexicanos,36.1
1,Aguascalientes,32.6
2,Baja California,48.4
3,Baja California Sur,42.8
4,Campeche,44.9
5,Coahuila,37.6
6,Colima,43.2
7,Chiapas,29.0
8,Chihuahua,38.7
9,Ciudad de México,36.3


In [ ]:
#Se unen los data frame, para tener la geometria y los datos de los estados en uno solo
geo=data_geo.merge(state_data,on="Entidad")

In [ ]:
#Se crea el mapa, añadiendo las coordenadas de la ubicación de México 
m = folium.Map(location=[24, -102], zoom_start=5, width='70%', height='70%',tiles='Stamen Watercolor')

#Se puede cambiar el estilo del mapa escogiendo entre:
#tiles= [OpenStreetMap, Mapbox Bright, Mapbox Control Room, Stamen Terrain, Stamen Toner, Stamen Watercolor, CartoDB positron, CartoDB dark_matter]


#Se crea el Mapa Coroplético, manejando el geojson de forma separada 

folium.Choropleth(
    geo_data=state_geo,
    name='Tasa de Obesidad',
    data=state_data,
    columns=['Entidad', 'Tasa de obesidad'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.4,
    legend_name='Tasa de Obesidad'
).add_to(m)


#Funciones para resaltar los estados al pasar el cursor
#Se define el color y la opacidad que tomara el estado al poner el cursor

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.7, 
                                'weight': 0.1}

#Se crea la función con el dataframe que contiene todos los datos
#Se selecciona la información a mostrar y las etiquetas, así como su estilo
NIL = folium.features.GeoJson(
    geo,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Entidad','Tasa de obesidad'],
        aliases=['Estado: ','Tasa de obesidad: '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(NIL)
m.keep_in_front(NIL)

#Se añade la opción para cambiar entre las capas del mapa
folium.LayerControl().add_to(m)

#Se añade un título al mapa
loc = 'Tasa de Obesidad'
title_html = '''
             <h3 style="font-size:20px"><b>{}</b></h3><br>
             '''.format(loc)
m.get_root().html.add_child(folium.Element(title_html))

#Se muestra el mapa
m

Podemos observar que son pocos los estados con una tasa de Obesidad baja, así como que los estados con mayor tasa se concentran al norte del país y en la península de Yucatan

#Mapa Porcentajes de Prevalencia de Enfermedades 2018

Se crea un mapa de México mostrando los porcentajes de la población que padece obesidad, diabetes e hipertensión por estado a partir de datos obtenidos del INEGI.

In [ ]:
#Se lee el csv con los porcentajes por estado
df=pd.read_csv('https://raw.githubusercontent.com/angyf/proyecto/main/Datos-porcentaje-poblacion-mayor-20.csv',index_col=0)

#Se cambian los nombres de los estados para que coincidan con los del geojson
df=df.replace({"Coahuila de Zaragoza": "Coahuila","Michoacán de Ocampo":"Michoacán","Veracruz de Ignacio de la Llave":"Veracruz", "Querétaro de Arteaga":"Querétaro"})

#Se crea un dataframe uniendo el geojson con el csv para tener toda la información de los estados en uno solo
geo_porc=data_geo.merge(df,on="Entidad")

In [ ]:
#Se crea el mapa, añadiendo las coordenadas de la ubicación de México
m = folium.Map(location=[24, -102], zoom_start=5, width='70%', height='100%',tiles='Stamen Watercolor')

#Se crean los Mapas Coropléticos, manejando el geojson de forma separada

#Layer o capa que muestra los porcentajes de Obesidad
folium.Choropleth(
    geo_data=state_geo,
    name='Obesidad',
    data=df,
    columns=['Entidad', 'Obesidad'],
    key_on='feature.properties.name',
    fill_color='YlGn',
    fill_opacity=1,
    line_opacity=0.4,
    legend_name='Obesidad'
).add_to(m)

#Layer o capa que muestra los porcentajes de Hipertensión
folium.Choropleth(
    geo_data=state_geo,
    name='Hipertensión',
    data=df,
    columns=['Entidad', 'Diagnóstico previo de hipertensión'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=1,
    line_opacity=0.4,
    legend_name='Hipertensión'
).add_to(m)

#Layer o capa que muestra los porcentajes de Diabetes
folium.Choropleth(
    geo_data=state_geo,
    name='Diabetes',
    data=df,
    columns=['Entidad', 'Diagnóstico previo de diabetes'],
    key_on='feature.properties.name',
    fill_color='RdPu',
    fill_opacity=1,
    line_opacity=0.4,
    legend_name='Diabetes'
).add_to(m)


#Se crea la función con el dataframe que contiene todos los datos
#Se selecciona la información a mostrar y las etiquetas, así como su estilo

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.7, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    geo_porc,
    style_function=style_function,
    overlay=True, 
    show=True,
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Entidad','Obesidad','Diagnóstico previo de hipertensión','Diagnóstico previo de diabetes'],
        aliases=['Entidad: ','Obesidad: ','Hipertensión: ','Diabetes: '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(NIL)
m.keep_in_front(NIL)

#Se añade un título
loc = 'Porcentajes de Prevalencia de enfermedades 2018.'
title_html = '''
            <h3 style="font-size:20px"><b>{}</b></h3><br>
             '''.format(loc)
m.get_root().html.add_child(folium.Element(title_html))

#Se quitan los botones de zoom para que no se empalmen con las leyendas
m.zoom_control=False
m.objects_to_stay_in_front

#Se añade la opción para cambiar entre las capas del mapa
folium.LayerControl().add_to(m)

#Se muestra el mapa
m

Se puede observar que la obesidad es la enfermedad que presenta porcentajes más altos

#Mapa Mortalidad de Enfermedades Cardiovasculares en el periodo 2013

Se crea un mapa mostrando la distribución de las defunciones por enfermedades cardiovasculares según datos del INEGI, así como la ubicación de los principales centros de salud especializados en cardiología.

In [ ]:
#Se lee el csv con las defunciones por estado
data=pd.read_csv('https://raw.githubusercontent.com/angyf/proyecto/main/Tasa_Mortalidad_Cardiovascular_Serie_Historica.csv')

#Se cambian los nombres de los estados para que coincidan con los del geojson
data=data.replace({"Coahuila de Zaragoza": "Coahuila", "Distrito Federal": "Ciudad de México","Michoacán de Ocampo":"Michoacán","Veracruz Llave":"Veracruz", "Querétaro de Arteaga":"Querétaro"})
data= data.rename(columns={'Entidad de Residencia':'Entidad'})

#Se aplica un filtro para quitar las defunciones totales a nivel nacional
data=data[data['Entidad']!="Nacional"]

In [ ]:
#Se aplica un filtro para obtener solo los datos más recientes 
data_2013=data[data['Periodo']==2013]

In [ ]:
#Se crea un dataframe uniendo el geojson con el csv para tener toda la información de los estados en uno solo
geo_2013=data_geo.merge(data_2013,on="Entidad")

In [ ]:
#Se crea el mapa, añadiendo las coordenadas de la ubicación de México
m = folium.Map(location=[24, -102], zoom_start=5, width='70%', height='70%')

#Se crea el layer que muestra las defunciones
folium.Choropleth(
    geo_data=state_geo,
    name='2013',
    data=data_2013,
    columns=['Entidad', 'Defunciones por Enfermedades Cardiovasculares'],
    key_on='feature.properties.name',
    fill_color='RdPu',
    fill_opacity=1,
    line_opacity=0.4,
    legend_name='Defunciones por Enfermedades Cardiovasculares'
).add_to(m)


#Se crea la función con el dataframe que contiene todos los datos
#Se selecciona la información a mostrar y las etiquetas, así como su estilo

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.7, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    geo_2013,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Entidad','Poblacion Total','Defunciones por Enfermedades Cardiovasculares', 'Tasa de Mortalidad por Enfermedades Cardiovasculares'],
        aliases=['Estado: ','Población: ','Defunciones: ','Tasa de Mortalidad: '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(NIL)
m.keep_in_front(NIL)


#Se añade el título
loc = 'Tasa de Mortalidad por Enfermedades Cardiovasculares 2013'
title_html = '''
             <h3 style="font-size:20px"><b>{}</b></h3><br>
             '''.format(loc)
m.get_root().html.add_child(folium.Element(title_html))


folium.LayerControl().add_to(m)
m

A continuación, se realiza el mismo mapa, mostrando esta vez la ubicación de los centros de salud

In [ ]:
#Se lee un csv que contiene el nombre del centro especializado en cardiología y su dirección
direcc=pd.read_csv('https://raw.githubusercontent.com/angyf/proyecto/main/Direcciones.csv')
direcc

,Centro,Dirección
0,"Centro Médico, ABC","Sur 136, No. 116. Col. Las Américas, Alvaro Ob..."
1,"Centro Médico Nacional 20 de Noviembre, ISSSTE","Av. Félix Cuevas, 540 3er piso. Col. del Valle..."
2,"Hospital Central Militar, SEDENA",Bvd. Manuel Ávila Camacho esq. Ejército Nacio...
3,"Hospital Central Sur de Alta Especialidad, PEMEX","Periferico Sur 4091, P. P. (e Investigación)...."
4,"Hospital Español de México, SBE","Av. Ejército Nacional No. 613, Torre mayor 8o ..."
5,"Hospital General de México, SS","Dr. Balmis No. 148, 1er piso Torre Gobierno. C..."
6,"Hospital Infantil de México, SS","Dr. Márquez 162. Col. Doctores, Cuauhtémoc 067..."
7,"Hospital Juárez de México, SS",Av. Instituto Politécnico Nacional No. 5160. C...
8,"Hospital Regional 1º de Octubre, ISSSTE",Av. Instituto Politécnico Nacional No.1669. Co...
9,"Instituto Nacional de Cardiología, SS","Juan Badiano No. 1. Col. Sección XVI, Tlalpan ..."


In [ ]:
#Se utiliza la librería de geopy para realizar la geocodificación y obtener las coordenadas a partir de la dirección
#Se utiliza la API de Bing Maps
from geopy.geocoders import Bing
geolocator = Bing("AhzzZWFo6COn8x2J_HbNv6eancFFyXmgCqU9LEJ5Emsk1yW2flEZpbF2-oMNgVxN")

In [ ]:
#Se obtienen las coordenadas de todas las direcciones
coordenadas=[]
for i in direcc.index: 
     location = geolocator.geocode(direcc['Dirección'][i])
     c=[location.latitude, location.longitude]
     coordenadas.append(c)

#location.address     

In [ ]:
#Se añade la columna con las coordenadas
direcc['Coordenadas']=coordenadas

In [ ]:
#Se crea el mapa de igual forma al anterior

m = folium.Map(location=[24, -102], zoom_start=5, width='70%', height='70%')

folium.Choropleth(
    geo_data=state_geo,
    name='2013',
    data=data_2013,
    columns=['Entidad', 'Defunciones por Enfermedades Cardiovasculares'],
    key_on='feature.properties.name',
    fill_color='RdPu',
    fill_opacity=1,
    line_opacity=0.4,
    legend_name='Defunciones por Enfermedades Cardiovasculares'
).add_to(m)


style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.7, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    geo_2013,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Entidad','Poblacion Total','Defunciones por Enfermedades Cardiovasculares', 'Tasa de Mortalidad por Enfermedades Cardiovasculares'],
        aliases=['Estado: ','Población: ','Defunciones: ','Tasa de Mortalidad: '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(NIL)
m.keep_in_front(NIL)


loc = 'Tasa de Mortalidad por Enfermedades Cardiovasculares 2013'
title_html = '''
             <h3 style="font-size:20px"><b>{}</b></h3><br>
             '''.format(loc)
m.get_root().html.add_child(folium.Element(title_html))


folium.LayerControl().add_to(m)


#Se añade un for que va añadiendo los marcadores, tomando las coordenadas y el nombre del dataframe
for i in direcc.index:
  folium.Marker(
    location=direcc['Coordenadas'][i],
    popup=direcc['Centro'][i],
    ).add_to(m)

m


Se puede observar que el mayor numero de defunciones por enfermedades cardiovasculares se concentra en el centro del pais, asimismo, es donde se cuenta con más centros de cardiología.